In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import sys
import tensorflow as tf
from sklearn.model_selection import train_test_split

### Load dataframe

In [2]:
main_data = pd.read_csv("data_labels_mainData.csv")
extra_data = pd.read_csv("data_labels_extraData.csv")

### split data into different folders for different cell types

In [7]:
import shutil
root = os.getcwd()
source_dir = os.path.join(root, "patch_images")

for i, row in main_data.iterrows():
    image_name = row["ImageName"]
    cell_name = row["cellTypeName"]
    is_cancer = str(row["isCancerous"])

    new_folder_multi = os.path.join(root, cell_name)
    new_folder_bin = os.path.join(root, is_cancer)

    os.makedirs(new_folder_multi, exist_ok=True)
    #os.makedirs(new_folder_bin, exist_ok=True)

    source = os.path.join(source_dir, image_name)

    dest_multi = os.path.join(root, cell_name)
    #dest_bin = os.path.join(root, is_cancer)

    go = shutil.copy(source,dest_multi)
    #go = shutil.copy(source, dest_bin)

### Add training data for cancerous classification from extra dataset 

In [4]:
import shutil
root = os.getcwd()
source_dir = os.path.join(root, "patch_images")

for i, row in extra_data.iterrows():
    image_name = row["ImageName"]
    is_cancer = str(row["isCancerous"])

    new_folder_bin = os.path.join(root, is_cancer)

    os.makedirs(new_folder_bin, exist_ok=True)

    source = os.path.join(source_dir, image_name)

    dest_bin = os.path.join(root, is_cancer)

    go = shutil.copy(source, dest_bin)

In [8]:
def get_images(img_list, path):    
    images = list()
    
    for img in img_list:
        img = cv2.imread(os.path.join(path, img))
        # resize to 0-1 for faster computation
        resized = img / 255
        images.append(resized)
        
    return images

images, cell_type, is_cancerous = get_images(main_data["ImageName"], "patch_images"), main_data[["cellType"]], main_data[["isCancerous"]]

print(len(images))
#print(images[0].shape)
print(len(cell_type))
print(len(is_cancerous))

9896
9896
9896


### Get data from splitted folder

In [17]:
def cancerous_classify_data():    
    images = list()
    labels = list()
    
    for label in [0, 1]:
        # get image directory
        img_dir = os.path.join(root, f"{label}")
        
        for img in os.listdir(img_dir):
            img = cv2.imread(os.path.join(img_dir, img))
            # resize to 0-1 for faster computation
            resized = img / 255
            images.append(resized)
            labels.append(label)
        
    return (images, labels)



In [18]:
images, cancer_label = cancerous_classify_data()

# try testing cancerous dataset
# x_train, x_test, y_train, y_test = train_test_split(
#     np.array(images), np.array(is_cancerous), test_size=0.2
# )

x_train, x_test, y_train, y_test = train_test_split(
    np.array(images), np.array(cancer_label), test_size=0.2
)

model = tf.keras.Sequential()

# First convo-pooling
# Convolutional layers (filter the image with a kernel)
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=[27, 27, 3]))
# Max-pooling layers (reduce the size of the image by choosing max pixel at certain area)
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
# Flatten input
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(32, activation="relu"))
# Output layer
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
# Compile model
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy", "AUC"])

print(x_train.shape)
print(y_train.shape)

model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
prediction = model.predict(x_test)

2022-04-29 13:32:00.256327: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-29 13:32:00.256641: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro
(16224, 27, 27, 3)
(16224,)
Epoch 1/10


2022-04-29 13:32:00.511445: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-29 13:32:00.709776: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


507/507 [==============================] - ETA: 0s - loss: 0.4174 - accuracy: 0.8073 - auc: 0.8752

2022-04-29 13:32:05.307202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


507/507 [==============================] - 6s 10ms/step - loss: 0.4174 - accuracy: 0.8073 - auc: 0.8752 - val_loss: 0.4552 - val_accuracy: 0.7894 - val_auc: 0.9141
Epoch 2/10
507/507 [==============================] - 5s 9ms/step - loss: 0.3507 - accuracy: 0.8465 - auc: 0.9146 - val_loss: 0.3302 - val_accuracy: 0.8568 - val_auc: 0.9240
Epoch 3/10
507/507 [==============================] - 5s 9ms/step - loss: 0.3316 - accuracy: 0.8574 - auc: 0.9239 - val_loss: 0.3199 - val_accuracy: 0.8614 - val_auc: 0.9323
Epoch 4/10
507/507 [==============================] - 5s 9ms/step - loss: 0.3219 - accuracy: 0.8611 - auc: 0.9285 - val_loss: 0.3130 - val_accuracy: 0.8651 - val_auc: 0.9322
Epoch 5/10
507/507 [==============================] - 5s 9ms/step - loss: 0.3021 - accuracy: 0.8743 - auc: 0.9372 - val_loss: 0.3479 - val_accuracy: 0.8526 - val_auc: 0.9351
Epoch 6/10
507/507 [==============================] - 5s 9ms/step - loss: 0.2974 - accuracy: 0.8742 - auc: 0.9390 - val_loss: 0.3021 - val_a

2022-04-29 13:32:49.016583: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
